# Disc Golf Coefficients Optimizer
#### This document calculates all the local maximum of disc golf throws using various inital conditions
#### The goal of this research is to see if mathmatical models of disc golf throws can simulate reality (throw 800+ ft). Current simulations seem to indicate that conditions only allow for shorter throws.

UPDATE: 4/13/2022 I got this to work! I found a combination of coeficients that will throw over 1000m. That was the goal of this script. I started to figure out how to add a timeout, but ran into some errors, so I gave up. The next steps are to record actual throws, then write a script that uses these principles to find the disc's coefficients given inital conditions and results.

In [1]:
from ipynb.fs.full.sharedfunctions import compute_custom_trajectory, Result, output_file_name
from ipywidgets import IntProgress

In [2]:
number_of_lines_written = 0
try:
    with open(output_file_name, 'r+') as file1:
        number_of_lines_written = len(file1.readlines())
    print(number_of_lines_written)
except:
    print('No file found')

No file found


In [3]:
def to_range(lower, upper=None, number_of_iterations_per_var=1, percent_range=None):
    if number_of_iterations_per_var == 1:
        return [lower]
    else:
        if percent_range is not None:
            upper = lower + lower*percent_range
            lower = lower - lower*percent_range
        else:
            if upper is None:
                upper = lower*-1

    return [lower + x*(upper-lower)/number_of_iterations_per_var for x in range(number_of_iterations_per_var+1)]

In [4]:
#PL0 = to_range(0.053,0.33,number_of_iterations_per_var=3)
#PLa = to_range(0.038,1.9,number_of_iterations_per_var=3)
#PD0 = to_range(0.055,0.18,number_of_iterations_per_var=3)
#PTy0 = to_range(-0.082,-0.007,number_of_iterations_per_var=3)
#PTya = to_range(0.003,0.43,number_of_iterations_per_var=3)

PL0 = to_range(0.33) # This is the best value
PLa = to_range(3.5625) # This is the best value
PD0 = to_range(0.055) # This is the best value
PTy0 = to_range(-0.002625) # This is the best value
PTya = to_range(0.003) # This doesn't do jack squat

#vx = to_range(10,120,number_of_iterations_per_var=4)
vx = to_range(60)
vy = to_range(0)
vz = to_range(0)
#phi = to_range(-0.36,number_of_iterations_per_var=2)
phi = to_range(-0.36) # This is the best value
theta = to_range(-0.14) # This is the best value
gamma = to_range(0)
dphi = to_range(0)
dtheta = to_range(0)
#dgamma = to_range(-62.0,number_of_iterations_per_var=2)
dgamma = to_range(-37.2) # This is the best value
#alpha_0  = to_range(-4*3.14/180)
alpha_0  = to_range(0.1395555555555556) # This is the best value
#PDa = to_range(0.69, number_of_iterations_per_var=5, percent_range=3)
PDa = to_range(-1.38)
PTxwx = to_range(-0.0143, number_of_iterations_per_var=5, percent_range=0.3)
PTxwz = to_range(-0.00153, number_of_iterations_per_var=5, percent_range=0.3)
PTywy = to_range(-0.01428, number_of_iterations_per_var=5, percent_range=0.3)
PTzwz = to_range(-0.0000374, number_of_iterations_per_var=5, percent_range=0.3)
area = to_range(0.058556)
I_zz = to_range(0.002352)
I_xx = to_range(0.001219)
mass = to_range(0.175)

In [5]:
all_vars = [
    PL0,
    PLa,
    PD0,
    PTy0,
    PTya,
    #x,
    #y,
    #z,
    vx,
    vy,
    vz,
    phi,
    theta,
    gamma,
    dphi,
    dtheta,
    dgamma,
    alpha_0,
    PDa,
    PTxwx,
    PTxwz,
    PTywy,
    PTzwz,
    area,
    I_zz,
    I_xx,
    mass,
]

In [6]:
def compute_custom_trajectory_optimizer(
        PL0,
        PLa,
        PD0,
        PTy0,
        PTya,
        x,
        y,
        z,
        vx,
        vy,
        vz,
        phi,
        theta,
        gamma,
        dphi,
        dtheta,
        dgamma,
        alpha_0 = 0,
        PDa = None,
        PTxwx = None,
        PTxwz = None,
        PTywy = None,
        PTzwz = None,
        area = None,  # m^2
        I_zz = None,  # kg*m^2
        I_xx = None,  # kg*m^2
        mass = None,  # kg
    ):
    result = compute_custom_trajectory(
        PL0,
        PLa,
        PD0,
        PTy0,
        PTya,
        x,
        y,
        z,
        vx,
        vy,
        vz,
        phi,
        theta,
        gamma,
        dphi,
        dtheta,
        dgamma,
        alpha_0 = alpha_0,
        PDa = PDa,
        PTxwx = PTxwx,
        PTxwz = PTxwz,
        PTywy = PTywy,
        PTzwz = PTzwz,
        area = area,
        I_zz = I_zz,
        I_xx = I_xx,
        mass = mass
    )
    if result is not None and len(result.x) > 0 and len(result.y) > 0:
        return result.x[-1], result.y[-1]
    return 0,0

In [7]:
def getResults(variables, values = []):
    var_values = variables[len(values)]
    for var_value in var_values:
        new_values = []
        for v in values:
            new_values.append(v)
        new_values.append(var_value)
        if len(new_values) == len(variables):
            if progressBar.value >= number_of_lines_written:
                #print(new_values)
                end_x, end_y = compute_custom_trajectory_optimizer(
                    new_values[0],
                    new_values[1],
                    new_values[2],
                    new_values[3],
                    new_values[4],
                    0, # x
                    0, # y
                    1, # z
                    new_values[5],
                    new_values[6],
                    new_values[7],
                    new_values[8],
                    new_values[9],
                    new_values[10],
                    new_values[11],
                    new_values[12],
                    new_values[13],
                    new_values[14],
                    new_values[15],
                    new_values[16],
                    new_values[17],
                    new_values[18],
                    new_values[19],
                    new_values[20],
                    new_values[21],
                    new_values[22],
                    new_values[23],
                )
                result = Result(new_values, end_x, end_y)
                print(end_x)
                with open(output_file_name, 'a') as f:
                    f.write(f'{result.toString()}\n')
            progressBar.value += 1
            if progressBar2.value >= 10:
                progressBar2.value = 0
            else:
                progressBar2.value += 1
        else:
            getResults(variables,new_values)

In [8]:
progressBar = IntProgress(min=0, max=0) # instantiate the bar
display(progressBar) # display the bar
progressBar2 = IntProgress(min=0, max=10) # instantiate the bar
display(progressBar2) # display the bar

IntProgress(value=0, max=0)

IntProgress(value=0, max=10)

In [9]:
max_count = 1
for var in all_vars:
    max_count = max_count*len(var)
progressBar.max = max_count
print(max_count)

1296


In [10]:
getResults(all_vars)

AttributeError: module 'signal' has no attribute 'SIGALRM'